In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import networkx as nx
plt.rcParams['figure.figsize'] = [15, 15]

Graph_data = nx.MultiDiGraph()

def print_graph(failed_names):
    color_map = []
    for node in Graph_data:
        if node in failed_names:
            color_map.append('blue')
            #print (node)
        else :
            color_map.append('red')
    nx.draw(Graph_data, with_labels=True,node_color = color_map, node_size=[v * 1000 for v in node_sizes.values()])
    plt.show()
    
thetas = np.arange(0.8,1.001,0.005)
Years = ['2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014']

cascade_result = pd.DataFrame()

for y in range(1,len(Years)):
    Country_matrix1 = pd.read_csv('TotalSectorWiseData' +Years[y] + '.csv')
    Country_matrix2 = pd.read_csv('TotalSectorWiseData' +Years[y-1] + '.csv')
    Country_matrix1 = Country_matrix1.set_index('Unnamed: 0')
    Country_matrix2 = Country_matrix2.set_index('Unnamed: 0')
    
    Country_matrix1 = Country_matrix1.drop(Country_matrix1.index[[476]])
    Country_matrix2 = Country_matrix2.drop(Country_matrix2.index[[476]])
    del Country_matrix2['RUS(Entertainment)']
    del Country_matrix1['RUS(Entertainment)']
    n = len(Country_matrix1)
    
    Countries = list(Country_matrix1)
    
    gdp_1 = sum(np.matrix(Country_matrix1))
    gdp_2 = sum(np.matrix(Country_matrix2))
    normalizing_value = max(gdp_1)
    
    gdp_2_normalized = [x / normalizing_value for x in gdp_2]
    gdp_1_normalized = [x / normalizing_value for x in gdp_1]
    fraction_gdp_loss = (gdp_2-gdp_1)/gdp_2
    
    fraction_ownership = np.matrix(Country_matrix1)/gdp_1
    C_hat = np.matrix(np.diag(np.diag(fraction_ownership)))
    C_matrix = fraction_ownership - C_hat
    A_matrix = np.dot(C_hat, np.linalg.inv((np.diag(np.ones(len(C_matrix))) - C_matrix)))
    
    p = np.matrix(gdp_1_normalized[0]).T
    
    theta_failled = []

    for theta in thetas:
        v_threshold = theta*(np.dot(A_matrix, gdp_2_normalized[0].T))

        pcurrent = p
    
        all_failure_indicator = (np.full((1, n), False)).astype(int)

        node_size = list(np.array(np.dot(A_matrix, pcurrent)).T[0])
    
        node_sizes = {}
        for i in range(len(node_size)):
            node_sizes[Countries[i]] = node_size[i]
        node_sizes
    
        for country in Countries:
            Graph_data.add_node(country)
    
        for country in Countries:
            for country2 in Countries:
                if country != country2:
                    Graph_data.add_weighted_edges_from([(country, country2, Country_matrix1[country][country2])])
                
        wave = 0
        wave_failled = []
        while (wave == 0) or (len(new_failed_names) != 0):
            wave = wave + 1
    
            new_failure_indicator = (np.array((np.dot(A_matrix, pcurrent).T < v_threshold.T).astype(int)) * ((all_failure_indicator == 0))).T
            all_failure_indicator = all_failure_indicator | new_failure_indicator.T
            pcurrent = pcurrent - np.multiply(new_failure_indicator, v_threshold/2)
            new_failed_countries = np.where(all_failure_indicator[0] == 1)[0]
            a = np.where(new_failure_indicator.T[0] == 1)
    
            node_size = list(np.array(np.dot(A_matrix, pcurrent)).T[0])

            node_sizes = {}
            for i in range(len(node_size)):
                node_sizes[Countries[i]] = node_size[i]
    
            failed_names = []
    
            for x in list(new_failed_countries):
                failed_names.append(Countries[x])
     
            new_failed_names = []
    
            for x in list(a[0]):
                new_failed_names.append(Countries[x])
            wave_failled.append(new_failed_names)
            
        theta_failled.append(wave_failled)
            
    cascade_result[Years[y]] = theta_failled
cascade_result.index = thetas
cascade_result.to_csv('Result.csv')

In [25]:
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
import networkx as nx
plt.rcParams['figure.figsize'] = [15, 15]

Graph_data = nx.MultiDiGraph()

In [78]:
Country_matrix14 = pd.read_csv('TotalSectorWiseData2014.csv')
Country_matrix12 = pd.read_csv('TotalSectorWiseData2013.csv')
Country_matrix14 = Country_matrix14.set_index('Unnamed: 0')
Country_matrix12 = Country_matrix12.set_index('Unnamed: 0')
Country_matrix14.head()

,AUS(Agricultural),AUS(Manufacturing),AUS(Mining),AUS(Power),AUS(Water),AUS(Retail),AUS(Transportation),AUS(Food),AUS(Entertainment),AUS(IT),...,ROW(Power),ROW(Water),ROW(Retail),ROW(Transportation),ROW(Food),ROW(Entertainment),ROW(IT),ROW(Finance),ROW(Education),ROW(Automobile)
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
AUS(Agricultural),13588.090783,5199.024283,122.549363,534.098308,2679.971937,3238.736075,2120.439248,180.693482,112.887843,150.731117,...,1.062591,16.718117,15.200032,74.873009,76.142623,5.900960,1.519048,34.344245,59.681608,18.748025
AUS(Manufacturing),28379.296358,38738.218419,24313.406554,7374.747167,2819.389322,13078.478399,14901.913058,1317.003570,854.645924,1659.816854,...,55.500607,118.810668,390.240738,775.420945,594.144021,79.944616,21.553076,212.755413,248.448469,55.759338
AUS(Mining),560.309594,8974.335993,4492.037652,3137.758321,7707.641785,4362.350024,4601.220504,1349.722417,370.799047,429.335837,...,10.696834,86.643642,86.250594,297.373241,611.399571,15.105845,5.421622,104.862540,233.400884,57.951465
AUS(Power),16.839482,1175.754135,998.395177,18182.814397,2966.595668,712.234618,751.730968,104.759146,65.619152,280.918040,...,5.889929,28.846039,13.512202,86.937123,45.837034,1.409978,2.343012,36.530058,46.426905,3.798949
AUS(Water),881.959436,45983.342844,2501.037942,1789.910818,111095.004218,13559.512535,10236.400956,1926.548898,1171.585891,3206.448231,...,11.927153,1049.375107,88.959063,300.552332,848.946128,64.030242,24.078863,168.646854,509.279321,112.070980


In [79]:
Country_matrix14.index[[476]]

Index(['RUS(Entertainment)'], dtype='object', name='Unnamed: 0')

In [80]:
Country_matrix14 = Country_matrix14.drop(Country_matrix14.index[[476]])
Country_matrix12 = Country_matrix12.drop(Country_matrix12.index[[476]])
del Country_matrix12['RUS(Entertainment)']
del Country_matrix14['RUS(Entertainment)']

In [81]:
Countries = list(Country_matrix14)

In [82]:
gdp_14 = sum(np.matrix(Country_matrix14))
gdp_12 = sum(np.matrix(Country_matrix12))
gdp_14.shape

(1, 571)

In [83]:
gdp_14[0,476]

39686.22874013931

In [84]:
normalizing_value = gdp_14.max()

In [85]:
gdp_12_normalized = [x / normalizing_value for x in gdp_12]
gdp_14_normalized = [x / normalizing_value for x in gdp_14]

In [86]:
Country_matrix14.shape

(571, 571)

In [87]:
fraction_ownership = np.matrix(Country_matrix14)/gdp_14
fraction_ownership.shape

(571, 571)

In [88]:
fraction_ownership[1,476]

3.019673061564309e-07

In [89]:
C_hat = np.matrix(np.diag(np.diag(fraction_ownership)))
C_hat

matrix([[0.22897385, 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.193814  , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.02788178, ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.17173231, 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.20568882,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.45454164]])

In [90]:
C_matrix = fraction_ownership - C_hat
C_matrix

matrix([[0.00000000e+00, 2.60116165e-02, 7.60655789e-04, ...,
         3.14466838e-05, 9.54926860e-05, 3.85737060e-05],
        [4.78221471e-01, 0.00000000e+00, 1.50911706e-01, ...,
         1.94805627e-04, 3.97526345e-04, 1.14723781e-04],
        [9.44181542e-03, 4.49001530e-02, 0.00000000e+00, ...,
         9.60154790e-05, 3.73449677e-04, 1.19234040e-04],
        ...,
        [2.11533458e-04, 1.06668701e-03, 1.89013217e-04, ...,
         0.00000000e+00, 4.08411633e-02, 4.06770659e-02],
        [9.26525273e-04, 2.74683664e-03, 8.89834832e-04, ...,
         1.22297995e-01, 0.00000000e+00, 7.64683029e-02],
        [1.82444818e-06, 4.27819784e-03, 3.46949559e-04, ...,
         1.96004150e-02, 2.38729547e-02, 0.00000000e+00]])

In [91]:
C_matrix[3,476]

5.5571760968810556e-08

In [92]:
A_matrix = np.dot(C_hat, np.linalg.inv((np.diag(np.ones(len(C_matrix))) - C_matrix)))
A_matrix

matrix([[2.35334796e-01, 1.07137280e-02, 2.33659071e-03, ...,
         1.81904786e-04, 1.93836642e-04, 1.70891857e-04],
        [1.06773136e-01, 2.13303356e-01, 3.43304172e-02, ...,
         9.01967806e-04, 8.64841129e-04, 7.52028336e-04],
        [2.00795150e-03, 2.83528895e-03, 2.84792570e-02, ...,
         6.11078019e-05, 6.64172011e-05, 5.59565219e-05],
        ...,
        [3.58825111e-03, 4.01619378e-03, 3.29411525e-03, ...,
         1.83063736e-01, 1.53967654e-02, 1.33131733e-02],
        [9.63387359e-03, 1.06614160e-02, 9.19074247e-03, ...,
         5.37212228e-02, 2.28299465e-01, 3.23909225e-02],
        [6.25571033e-03, 7.64206093e-03, 6.90202284e-03, ...,
         2.89792941e-02, 2.35703013e-02, 4.66839415e-01]])

In [97]:
# A_matrix.sum(axis = 0)